# Test notebook

__A. J. Tropiano [tropiano.4@osu.edu]__<br/>
__March 17, 2022__

The purpose of this notebook is for testing out code and/or scripts.

_Last update: April 1, 2022_

In [1]:
# Python imports
from matplotlib.offsetbox import AnchoredText
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import time

In [2]:
# Imports from A.T. codes
import densities
from momentum_distributions import MomentumDistributions
from vnn import Potential
# You could import modules but I want a list of scripts here
# Check these off the list as you go through Jupyter notebooks generating
# figures
# import dmd, snmd, pmd
import modules.a2
import modules.figures
import modules.fourier_transform
import modules.integration
import modules.labels
import modules.long_distance_operators
from modules.magnus import Magnus_split
import modules.momentum_projection_operator
import modules.phase_shifts
import modules.sdt
from modules.srg import get_transformation
import modules.tools
import modules.wave_function
# add missing ones

For now let's illustrate the overhaul by testing scripts and showing how to use them.

### Test `vnn.py`

In [ ]:
# kvnn = 900
# channel = '3S1'
# kmax, kmid, ntot = 30.0, 4.0, 120

In [3]:
# For comparing to old code
from os import getcwd, chdir

cwd = getcwd()
old_code_dir = '../../../tropiano-srg-old'

chdir(old_code_dir)

from run_srg import run_srg as rs_old
from potentials.vsrg_macos import vnn as vnn_old
from snmd import single_nucleon_momentum_distributions
from pmd import pair_momentum_distributions
from dmd import deuteron_momentum_distributions

chdir(cwd)

In [ ]:
# # __init__ (check)

# potential = Potential(kvnn, channel, kmax, kmid, ntot)
# print(potential.kvnn, potential.coupled_channel_bool, 
#       potential.potential_directory)

In [ ]:
# # load_mesh (check)

# k_array, k_weights = potential.load_mesh()
# for ik, iw in zip(k_array, k_weights):
#     print(ik, iw)

In [ ]:
# # load_potential (check)

# V_matrix = potential.load_potential()
# print(V_matrix)

In [ ]:
# # load_hamiltonian (check)

# H_matrix = potential.load_hamiltonian()
# print(H_matrix)

In [ ]:
# # Compare to Hamiltonian using vnn.py from tropiano-srg-old (check)

# chdir(old_code_dir)

# H_matrix_old = vnn_old.load_hamiltonian(kvnn, channel, kmax, kmid, ntot)
# print(H_matrix_old)

# chdir(cwd)

### Test `srg.py`

In [ ]:
# # run_srg for Wegner (check)

# generator = 'Wegner'
# lambda_array = np.array([6.0, 3.0])

# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [ ]:
# chdir(old_code_dir)

# d_old = rs_old(kvnn, channel, kmax, kmid, ntot, generator, lambda_array,
#                save=False)
# print(d_old[3.0])

# chdir(cwd)

In [ ]:
# # run_srg for T (check)

# generator = 'T'

# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [ ]:
# chdir(old_code_dir)

# d_old = rs_old(kvnn, channel, kmax, kmid, ntot, generator, lambda_array,
#                save=False)
# print(d_old[3.0])

# chdir(cwd)

In [ ]:
# # run_srg for Block-diag (check)

# generator = 'Block-diag'
# lambda_bd_array = np.array([6.0, 3.0])
# lambda_array = np.array([1.0])

# d = potential.run_srg(generator, lambda_array, lambda_bd_array, save=False)
# print(d[3.0][1.0])

In [ ]:
# chdir(old_code_dir)

# d_old = rs_old(kvnn, channel, kmax, kmid, ntot, generator, lambda_array,
#                lambda_bd=lambda_bd_array[-1], save=False)
# print(d_old[1.0])

# chdir(cwd)

In [ ]:
# # compare evolved Hamiltonian using get_transformation (check)

# H_initial = potential.load_hamiltonian()
# H_evolved = potential.load_hamiltonian('srg', 'Wegner', 3.0)
# U_matrix = get_transformation(H_initial, H_evolved)
# print(U_matrix @ H_initial @ U_matrix.T / potential.hbar_sq_over_m)

### Test `magnus.py`

In [ ]:
# # Might be smart to test a really soft potential here for speed
# kvnn = 111
# channel = '1S0'
# kmax, kmid, ntot = 10.0, 2.0, 120

# potential = Potential(kvnn, channel, kmax, kmid, ntot)

In [ ]:
# # Do default Magnus arguments
# k_max = 6
# ds = 1e-5

In [ ]:
# # run_magnus for Wegner (check)

# generator = 'Wegner'
# lambda_array = np.array([6.0, 3.0])

# d = potential.run_magnus(generator, lambda_array, k_max=k_max, ds=ds,
#                          save=False)
# print(d[3.0])

# # run_srg for Wegner
# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [ ]:
# # run_magnus for T (check)

# generator = 'T'
# lambda_array = np.array([6.0, 3.0])

# d = potential.run_magnus(generator, lambda_array, k_max=k_max, ds=ds,
#                          save=False)
# print(d[3.0])

# # run_srg for Wegner
# d = potential.run_srg(generator, lambda_array, save=False)
# print(d[3.0])

In [ ]:
# # run_magnus for Block-diag

# generator = 'Block-diag'
# lambda_bd_array = np.array([6.0, 3.0])
# lambda_array = np.array([3.0])

# d = potential.run_magnus(generator, lambda_array, lambda_bd_array, 
#                          k_max=k_max, ds=ds, save=False)
# print(d[3.0][3.0])

# # run_srg for Block-diag (check)
# d = potential.run_srg(generator, lambda_array, lambda_bd_array, save=False)
# print(d[3.0][3.0])

In [ ]:
# # Test that Magnus_split at least works (check)

# generator = 'Wegner'
# lambda_array = np.array([6.0, 3.0])
# k_max = 6
# ds = 1e-5

# magnus_split = Magnus_split(potential, generator)
# d = magnus_split.magnus_split_evolve(lambda_array, k_max=k_max, ds=ds)
# print(d[3.0])

### Testing momentum distributions code

In [4]:
# kvnn = 6
# kmax, kmid, ntot = 15.0, 3.0, 120
kvnn = 111
kmax, kmid, ntot = 10.0, 2.0, 120
channels = ('1S0', '3S1')
generator = 'Wegner'
lamb = 1.5

In [5]:
# This works comparing to old snmd.py and pmd.py
md = MomentumDistributions(kvnn, kmax, kmid, ntot)
md.save_deltaU_funcs(channels, generator, lamb)

[[-1.71605041e-02  3.56553624e+02  5.61839838e+01 ...  2.43823509e-06
   2.31015997e-06  5.65828738e-07]
 [-3.56581559e+02 -1.41440188e-02  6.48341599e+01 ...  2.43787829e-06
   2.30981982e-06  5.67278256e-07]
 [-5.62090640e+01 -6.48604344e+01 -1.33295764e-02 ...  2.43740163e-06
   2.30935453e-06  5.76054818e-07]
 ...
 [-2.99155373e-06 -2.99156423e-06 -2.99161875e-06 ... -3.65329715e-06
   2.25681072e-04  3.41614727e-04]
 [-2.88302365e-06 -2.88303105e-06 -2.88306998e-06 ... -2.32730761e-04
  -3.55246981e-06  1.25995888e-04]
 [-1.14992723e-06 -1.15167084e-06 -1.16087990e-06 ... -3.48444236e-04
  -1.33034690e-04 -3.52572419e-06]]
[[-1.89494541e-01  3.68085747e+02  5.84289711e+01 ...  5.87256028e-06
   5.64686128e-06  1.29319167e-05]
 [-3.68444746e+02 -1.79641270e-01  6.88255034e+01 ...  5.87239262e-06
   5.64668278e-06  1.29232128e-05]
 [-5.87859774e+01 -6.91833639e+01 -1.79076147e-01 ...  5.87203597e-06
   5.64624315e-06  1.28785738e-05]
 ...
 [-4.93303689e-06 -4.93304821e-06 -4.9331069

In [6]:
chdir(old_code_dir)
snmd = single_nucleon_momentum_distributions(kvnn, channels, lamb, kmax, kmid,
                                             ntot)
chdir(cwd)

[[-1.71602283e-02  3.56553563e+02  5.61839746e+01 ...  2.43823506e-06
   2.31016000e-06  5.65828753e-07]
 [-3.56581498e+02 -1.41440231e-02  6.48337920e+01 ...  2.43787896e-06
   2.30981962e-06  5.67277204e-07]
 [-5.62090549e+01 -6.48600664e+01 -1.33295762e-02 ...  2.43740180e-06
   2.30935449e-06  5.76054800e-07]
 ...
 [-2.99155372e-06 -2.99156397e-06 -2.99161882e-06 ... -3.65329715e-06
   2.25681072e-04  3.41614727e-04]
 [-2.88302362e-06 -2.88303146e-06 -2.88307013e-06 ... -2.32730761e-04
  -3.55246981e-06  1.25995888e-04]
 [-1.14992725e-06 -1.15167122e-06 -1.16088001e-06 ... -3.48444236e-04
  -1.33034690e-04 -3.52572420e-06]]
[[-1.89565979e-01  3.68085793e+02  5.84289667e+01 ...  5.87256026e-06
   5.64686129e-06  1.29319167e-05]
 [-3.68444793e+02 -1.79641276e-01  6.88252409e+01 ...  5.87239243e-06
   5.64668255e-06  1.29232127e-05]
 [-5.87859730e+01 -6.91831014e+01 -1.79076148e-01 ...  5.87203594e-06
   5.64624309e-06  1.28785739e-05]
 ...
 [-4.93303687e-06 -4.93304780e-06 -4.9331069

In [7]:
md.save_deuteron_deltaU_funcs(generator, lamb)

[[-6.29080344e-02  6.32618375e+01  1.01126214e+01 ...  1.55116836e-06
   1.49728071e-06  4.21635510e-06]
 [-6.33768482e+01 -5.75230870e-02  1.21360100e+01 ...  1.55115079e-06
   1.49725732e-06  4.21321829e-06]
 [-1.02274434e+01 -1.22509178e+01 -5.74704551e-02 ...  1.55110877e-06
   1.49718598e-06  4.19701008e-06]
 ...
 [-1.14575335e-06 -1.14575539e-06 -1.14576579e-06 ... -7.93162990e-06
   4.68792278e-05  7.48166088e-05]
 [-1.10690933e-06 -1.10690491e-06 -1.10688245e-06 ... -6.21480030e-05
  -7.70756617e-06  2.30456466e-05]
 [-3.83271130e-06 -3.82959237e-06 -3.81344213e-06 ... -8.95698209e-05
  -3.83103360e-05 -7.65048185e-06]]


In [8]:
chdir(old_code_dir)
dmd = deuteron_momentum_distributions(kvnn, lamb, kmax, kmid, ntot)
chdir(cwd)

[[-6.29549930e-02  6.32691774e+01  1.01121699e+01 ...  1.55116845e-06
   1.49728080e-06  4.21635520e-06]
 [-6.33841882e+01 -5.75230838e-02  1.21361208e+01 ...  1.55115061e-06
   1.49725727e-06  4.21321840e-06]
 [-1.02269920e+01 -1.22510286e+01 -5.74704549e-02 ...  1.55110874e-06
   1.49718597e-06  4.19701013e-06]
 ...
 [-1.14575335e-06 -1.14575530e-06 -1.14576579e-06 ... -7.93162990e-06
   4.68792278e-05  7.48166088e-05]
 [-1.10690933e-06 -1.10690514e-06 -1.10688249e-06 ... -6.21480030e-05
  -7.70756617e-06  2.30456467e-05]
 [-3.83271136e-06 -3.82959260e-06 -3.81344218e-06 ... -8.95698209e-05
  -3.83103361e-05 -7.65048185e-06]]


In [ ]:
# Test figure_graphics.py

In [ ]:
# Test figure_labels.py

In [ ]:
# Test fourier_transform.py

In [ ]:
# Test integration.py

In [ ]:
# Test long_distance_operators.py

In [ ]:
# Test momentum_projection_operator.py

In [ ]:
# Test phase_shifts.py

In [ ]:
# Test sdt.py

In [ ]:
# Test srg.py

In [ ]:
# Test tools.py

In [ ]:
# Test wave_function.py